<a href="https://colab.research.google.com/github/alpha-davidson/ICTP-IAEA-Workshop-2022/blob/main/Intro_to_NN_with_TF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Networks in TensorFlow's Keras


We will use a dense neural network in Keras to solve a simple regression problem. 
## Learning Task
We will construct a dense neural network to predict the invariant mass of a particle from its energy, momentum, (and charge).

*Note that this task does not require machine learning. We choose a task with a known mapping to help us create, debug, and tune our first neural network.*

## Dataset
This dataset is a collection of simulated particle events from [Pythia](http://home.thep.lu.se/~torbjorn/Pythia.html). The dataset is a 2D array where each row represents one event from an $e^{-} + p$ collision. This dataset is comprised _only_ of events where exactly 16 particles are produced from an electron-proton collision. Each particle contains $(p_x,p_y,p_z,E,q)$. Each event is therefore represented by 80 numbers. 

**Advanced activity:** There are more interesting event-wise learning tasks using this dataset. Consider crafting your own learning task and target for this data.



## Computational Notes

If this is your first time in a Jupyter-like environment, please read the following carefully:

 - You are in an active kernel
 - Run each cell with `Shift + Enter`
 - You must execute the cells in the order that you want the code to run
 - In Colab, `Runtime`$\rightarrow$`Change runtime type` allows you to utilize GPUs and TPUs. They are unnecessary here, but will become vital in later exercises.


In [ ]:
# import the packages we will be using
import numpy as np
import tensorflow as tf
import pylab as plt

In [ ]:
# Import data from github. Note: in colab, go to Files and refresh to see file.
# Here I use Linux commands within the notebook to pull the data file and rename it.

# If you are not in an environment that supports the following commands, 
#     paste the url below to download the dataset onto your local machine.

!wget https://github.com/NuclearTalent/MachineLearningECT/blob/master/doc/pub/Day6/data/homogenous-16-particle-events-energy.npy?raw=true
!mv homogenous-16-particle-events-energy.npy?raw=true particle-events.npy

In [ ]:
# now we load the data file, which is a numpy array
events = np.load("particle-events.npy")

Recall that each row of this dataset is an entire event. We need each row to represent a training example, which is a single particle.

Using `numpy`'s `reshape` method we can make each row represent one particle. 

In [ ]:
# Here we rearrange the data within each of the events to isolate particles

#print("events[0] =\n",events[0])
evt_particles = np.reshape(events, (len(events), 16, 5))

#print("\nevt_particles[0] =\n", evt_particles[0])

# Use another call of reshape to combine all events to have the appropriate shape
# Complete me:
particles = 

#print("\nparticles =\n",particles[:10])

These are our training data inputs, but we also must provide the targets, which are the invariant masses of each particle. This is a straightforward computation.

We choose units where $c = 1$:
$$m^2=E^2-||\textbf{p}||^2$$
where $m, E$, and $\textbf{p}$ are all in GeV.

**Create an array of your target values.**
Due to insufficient precision, some $m^2$ values for massless particles will come out very slightly negative. These should be treated as zero to avoid `NaN`. I used the `maximum` method from `numpy` to handle this. 

In [ ]:
# Complete me:


There are several hundred thousand datapoints in this dataset which is overkill for this simple example. Create a test dataset with just 1000 examples.

In [ ]:
#Slicing allows you select a subset of an array.
#This can be done like this: smallerArray = largerArray[:100]

#Complete me:


Next, make a histogram of the target data to make sure that we are seeing masses of real particles. As this data has limited precision, this will not resolve electrons very well, but protons, pions, and massless particles should be clearly visible.

In [ ]:
plt.hist(mass_train,bins=100)
plt.show()

Now we can build and train the first neural network. We will start with a network with on hidden layer and 5 neurons, and ReLU activation. 




In [ ]:
# Build our model
model = tf.keras.Sequential() #Define the model object
model.add(tf.keras.layers.Dense(5, input_shape=(5,), activation="relu")) #Add a single hidden layer

# Add the output layer yourself
# No activation function assignment defaults to "linear"
# Complete me:



# prints a model summary
model.summary()

To start, train for 30 epochs with a batch size of 128, an Adam optimizer with a learning rate of 0.1, using mean squared error loss. **This is not ideal.**

Define your validation split to provide a set of data the model has not used to assess how the model is generalizing. This allows us to monitor overtraining. Setting `validation_split=0.2` meaning that $20\%$ of the data will be used for validation. **This is not ideal.**

Information on how to implement these features can be found here:
<https://www.tensorflow.org/api_docs/python/tf/keras>.
Check out Sequential underneath models and Dense under layers.

In [ ]:
optimize = tf.keras.optimizers.Adam(lr=0.1) # Adam optimizer with a learning rate of 0.1
loss_func = tf.keras.losses.MeanSquaredError() # choosing MSE

# compile model
model.compile(optimizer=optimize,loss=loss_func) 

In [ ]:
results = model.fit(particles_train, mass_train, epochs=10, batch_size=256, validation_split=0.2)

A learning curve is a clearer way to visualize the above training. This helps us determine over and underfitting, which, in turn, helps us tune our model and training.

In [ ]:
# helper function for easy plotting
def plot_learning_curve(history):
    plt.plot(history["loss"], label="training loss")
    plt.plot(history["val_loss"], label="validation loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.yscale('log')
    plt.legend()
    plt.show()


In [ ]:
# using the new function
plot_learning_curve(results.history)

## Tuning
The model is learning, but we can do better. 

**At this point, you can play with the above code to change hyperparameters and note changes to your results.**

Alternatively, you can step through tuning this model below.

## Walk-through tuning:
Perhaps we did not have enough model parameters to accurately represent the mapping. Remedy this by increasing the number of hidden neurons to 20.

In [ ]:
#Use the same code as in the previous cell
#simply change the number of neurons in the hidden layer

We see that we got little improvement here. Another hyperparameter to adjust is *batch size*, which is the number of training examples used to calculate the gradient on each step. While you may initially think that a higher batch size leads to faster or more accurate training, in practice this is not true. The "noise" that arises from using less training examples at each iteration can actually help find the global minimum of the loss function.
(See here for more info: https://arxiv.org/pdf/1609.04836.pdf)

Try decreasing the batch size to 16.

In [ ]:
#Complete me:


This is starting to do better but has significant room for improvement.

Another hyperparameter to tune is the *learning rate*. 

 - If the learning rate is too high, we are taking too large of a step in the gradient descent at each iteration and will miss narrow minima in the loss function. 
 - If the learning rate is too small, then we are not traveling far enough in each iteration and we will take far too long to reach a minimum. 

Perhaps the learning rate is too high and the network can't fine tune. Try decreasing the learning rate to 0.001.

In [ ]:

# Complete me:


This is not really that much better, but now there is evidence of *overtraining* or *overfitting* -- the training loss is so much lower than the validation loss. 

A common fix to this is adding *dropout layers*. Try adding a dropout layer with dropout rate of 0.5. <https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout>

You can also try batch normalization: <https://www.tensorflow.org/api_docs/python/tf/keras/layers/BatchNormalization>

In [ ]:
#Dropout layers are located under tf.keras.layers. 
#They take the dropout rate as their only argument.
#BatchNormalization layers are also under tf.keras.layers, and in the simplest use case, take no arguments

#Complete me:


This clearly stopped the overtraining problem, but it still isn't training well. Now, try training on the full dataset with a more reasonable validation split of 0.2. Use a single hidden layer with 20 neurons, a learning rate of 0.001, and a batch size of 256. Just run it for 10 epochs.

In [ ]:
#Complete me:


This clearly resulted in a significant improvement and shows how important having a large enough dataset is. Moving on to the choice in activation functions, ReLU is not the only available choice, although it is one of the most popular ones currently. Try training a network using a sigmoid or tanh activation function.

In [ ]:
#Simply change relu to sigmoid or tanh to change the activation function

#Complete me:


Next, try adding 2 new hidden layers to the network. Use the ReLU activation function.

In [ ]:
#Complete me:


Clearly, adding more layers helps improve the quality of the network. There is a limit to how effective this is though. Try having 5 hidden layers.

In [ ]:
#Complete me:


Now, see what happens when you increase the number of neurons per layer from 20 to 50 in the 3 hidden layer model. Consider how they perform compared to ReLU now. 

In [ ]:
#Complete me:


Try using the sigmoid and the tanh activation functions again and compare them to ReLU.

In [ ]:
#Complete me:


This difference in performance, especially with the sigmoid function, is known as the vanishing gradient problem. If the value for any one the neurons gets too far away from 0, the gradient for sigmoid and tanh gets really close to 0. This means that for deeper networks it is much more difficult to update the weights in the earlier layers as their gradient is so small. Now, remove the fifth column from the input data, the charge, and see what happens when training. Why do you think including charge has such a large impact, even though it is not required to compute invariant mass?

In [ ]:
#Complete me:


Finally, there are other options for the loss function. Try experimenting with alternatives to mean squared error.

<https://www.tensorflow.org/api_docs/python/tf/keras/losses>

You can also try some other optimizers -- for example, sgd (with and without momentum), rmsprop, adagrad, adadelta, adamax, and nadam. <https://www.tensorflow.org/api_docs/python/tf/keras/optimizers>

